### load models and infer the classification from new imagens

In [3]:
import os
import random
from PIL import Image
import shutil

import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model
from tensorflow.keras.preprocessing import image
import plotly.express as px

In [2]:
folder_path = 'frames'
num_images_to_select = 200

In [18]:

all_images = os.listdir(folder_path)

if len(all_images) < num_images_to_select:
    print("Error: Not enough images in the folder.")
else:
    selected_indices = random.sample(range(len(all_images)), num_images_to_select)

    selected_folder_path = 'resized_frames'
    os.makedirs(selected_folder_path, exist_ok=True)

    for index in selected_indices:
        image_name = all_images[index]
        image_path = os.path.join(folder_path, image_name)

        img = Image.open(image_path)

        img_resized = img.resize((150, 150))

        resized_image_path = os.path.join(selected_folder_path, image_name)
        img_resized.save(resized_image_path)

200 random images have been successfully selected, resized to 150x150 pixels, and copied to resized_frames.


In [76]:
model = load_model('h5_jetson_nano_copy/RESNETV2-2023-09-05_03-07-59-2023-09-05_06-13-27.h5')

In [77]:
classes = ['Frente+Umida', 'Fundo+Intermediaria', 'Meio+Seca', 'camendoim6sCenL1'] 

In [ ]:
image_directory = 'resized_frames'
class_name = 'camendoim6sCenL1'
results = []

# Create a DataFrame to store the results
columns = ['Filename'] + classes
df_resnet = pd.DataFrame(columns=columns)

for filename in os.listdir(image_directory):
    if filename.endswith('.png'):
        img_path = os.path.join(image_directory, filename)
        
        img = plt.imread(img_path)
        img = cv2.resize(img, (150, 150), interpolation=cv2.INTER_AREA)
        img = np.expand_dims(img, 0)
        img = img / 255.0 
        
        predictions_single = model.predict(img)

        # Append results to the list
        results.append((filename,) + tuple(predictions_single[0]))

        # Append results to the DataFrame
        row_data = {'Filename': filename}
        row_data.update({class_name: prob for class_name, prob in zip(classes, predictions_single[0])})
        df_resnet = df_resnet.append(row_data, ignore_index=True)

# Print the DataFrame with the results
# print(df.head())

In [ ]:
csv_filename = 'resnet50v2-mod_predictions.csv'
df_resnet.to_csv(csv_filename, index=False)

In [38]:
csv_filename = 'resnet50v2-mod_predictions.csv'
df = pd.read_csv(csv_filename)

df.rename(columns={'Frente+Umida':'Frente úmida (maravalha)', 'Fundo+Intermediaria':'Fundo intermediária (maravalha)', 'Meio+Seca':'Meio semi-seca (maravalha)', 'camendoim6sCenL1':"Casca de amendoim"}, inplace=True)

df_melted = pd.melt(df, value_vars=df.columns[1:], var_name='Class', value_name='Probability')

fig = px.violin(df_melted, x='Class', y='Probability', color='Class',
                labels={'Probability': 'Probabilidade', 'Class': 'Classe'},
                )

fig.update_layout(height=800, width=2000)
fig.update_layout(yaxis=dict(range=[0, 1]))
fig.update_layout(
    plot_bgcolor='white',
    font=dict(family='Times', size=36, color='black')  
)
fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    showgrid=False 
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    showgrid=False  
)

fig.update_traces(showlegend=False)

fig.show()

In [47]:
# MODEL RESNET
print(count, len(results), count / len(results))

200 200 1.0
